Latex Macros:
$$
\newcommand{\E}{\text{E}}
\newcommand{\mbf}{\mathbf}
\newcommand{\bs}{\boldsymbol}
\newcommand{\Cov}{\text{Cov}}
\newcommand{\Var}{\text{Var}}
\newcommand{\Cov}{\text{Cov}}
\newcommand{\A}[1]{\mathbf{A}_{#1}}
\newcommand{\Ai}[1]{\mathbf{A}^{#1}}
$$

# Inference on difference beteween the most favorable and least unfavorable genotypic values  

Consider the model:
$$
\mbf{y} = \mbf{1}\bs{\mu} + \mbf{M}\bs{\alpha} + \mbf{e},
$$

where $\mbf{M}$ is a matrix of SNP covariates coded as 0, 1 or 2 (number of copies of the "A" allele). At locus $j$, if the "A" allele is favorable, $\alpha_j$ is positive, and if it is unfavorable,$\alpha_j$ is negative. So, if locus $j$ is favorable, the difference between the favorable and unfavorable homozygotes will be $2 \times \alpha_j$. On the other hand, if locus $j$ is unfavorable, this difference becomes $2\times -\alpha_j$. Thus, at an arbitrary locus, the difference between the favorable and unfavorable homozygotes is $2 \times |\alpha_j|$, and across all the loci, the difference between the most favorable and least favorable genotypic values is:

$$
D = 2\sum_j |\alpha_j|.
$$

But, as the $\bs{\alpha}$ is not observed, $D$ can be estimated as
$$
D = 2\sum_j |\hat{\alpha}_j|.
$$

Below are results from several estimators of $\bs{\alpha}$. 

In [4]:
using Statistics, Distributions, JWAS, LinearAlgebra, Printf, Plots, CSV, DataFrames
using Random

Here is a simple way to define a function:

In [6]:
Identity(n) = Matrix(I,n,n)

Identity (generic function with 1 method)

In [12]:
function getEstimates(X,mmei,xpxi,probNull, Vg, Ve)
    n,p = size(X)
    pNull = Int(probNull*p)
    pNonNull = p - pNull
    Valpha = 2Vg/p
    α = [randn(pNonNull)*sqrt(Valpha); zeros(pNull)]
    y = X*α + randn(n) .* sqrt(Ve)
    data = DataFrame([string.(1:n) y])
    names!(data,[:ind, :y])
    
    model = build_model("y = intercept", Ve)
    add_genotypes(model,"genotypes", Vg; G_is_marker_variance=false, separator=',',header=false)
    
    ### Bayes A ###
    outputA = runMCMC(model,data;
                Pi                  = 0.0,
                burnin              = 1000,
                chain_length        = 11000,
                starting_value      = false,
                missing_phenotypes  = false,
                constraint          = false,
                estimatePi          = false,
                estimate_variance   = true,
                methods             = "BayesB",
                printout_frequency  = 5_000,
                printout_model_info = false,
                output_samples_frequency = 0,
                update_priors_frequency=0,
                #parameters for single-step analysis
                single_step_analysis= false,
                pedigree            = false,
                #output
                #Genomic Prediction
                outputEBV               = false,
                output_genetic_variance = true,
                output_residual_variance= true,
                output_heritability     = false
    )
    
    ### Bayes BPi ###
    model = build_model("y = intercept", Ve)
    add_genotypes(model,"genotypes", Vg; G_is_marker_variance=false, separator=',',header=false)
    outputB = runMCMC(model,data;
                Pi                  = 0.5,
                burnin              = 1000,
                chain_length        = 11000,
                starting_value      = false,
                missing_phenotypes  = false,
                constraint          = false,
                estimatePi          = true,
                estimate_variance   = true,
                methods             = "BayesB",
                printout_frequency  = 5_000,
                printout_model_info = false,
                output_samples_frequency = 0,
                update_priors_frequency=0,
                #parameters for single-step analysis
                single_step_analysis= false,
                pedigree            = false,
                #output
                #Genomic Prediction
                outputEBV               = false,
                output_genetic_variance = true,
                output_residual_variance= true,
                output_heritability     = false
    )
    
    ### Bayes CPi ###
    model = build_model("y = intercept", Ve)
    add_genotypes(model,"genotypes", Vg; G_is_marker_variance=false, separator=',',header=false)
    outputC = runMCMC(model,data;
                Pi                  = 0.5,
                burnin              = 1000,
                chain_length        = 11000,
                starting_value      = false,
                missing_phenotypes  = false,
                constraint          = false,
                estimatePi          = true,
                estimate_variance   = true,
                methods             = "BayesC",
                printout_frequency  = 5_000,
                printout_model_info = false,
                output_samples_frequency = 0,
                update_priors_frequency=0,
                #parameters for single-step analysis
                single_step_analysis= false,
                pedigree            = false,
                #output
                #Genomic Prediction
                outputEBV               = false,
                output_genetic_variance = true,
                output_residual_variance= true,
                output_heritability     = false
    )
    rhs = X'y
    ### RR-BLUP ###
    blup = mmei*rhs
    
    ### OLS ###
    ols  = xpxi*rhs
    
    bayesa = outputA["Posterior mean of marker effects"]
    bayesb = outputB["Posterior mean of marker effects"]
    bayesc = outputC["Posterior mean of marker effects"]
    trueMean   = 2*sum(abs.(α))
    bayesaMean = 2*sum(abs.(bayesa))
    bayesbMean = 2*sum(abs.(bayesb))
    bayescMean = 2*sum(abs.(bayesc))
    blupMean   = 2*sum(abs.(blup))
    olsMean    = 2*sum(abs.(ols))    
    return trueMean,bayesaMean,bayesbMean,bayescMean,blupMean,olsMean
end

getEstimates (generic function with 1 method)

### Comparison of BayesA, BayesB, BayesCPi, RR-BLUP and OLS with Sample size n = 100 and number p = 20 of markers and $\pi = 0.75$

In [13]:
n  = 100
p  = 20
probNull = 0.75

N  = 25
Vg = 20
Ve = 10

Random.seed!(1234)

Valpha = Vg/p*2
λ = Ve/Valpha
X = rand(Binomial(2,0.5), n,p)
mme = float(X'X + Identity(p)*λ)
mmei = inv(mme)
xpxi = inv(X'X)
CSV.write("genotypes",DataFrame([string.(1:n) X]), writeheader=false)

resSim = [getEstimates(X,mmei,xpxi,probNull,Vg, Ve) for i=1:N]
resMat = [row[i] for row in resSim, i=1:6]

resMeans = mean(resMat,dims=1)
resStd = std(resMat,dims=1)./N;

20 markers on 100 individuals were added.

The prior for marker effects variance is calculated from the genetic variance and π. The mean of the prior for the marker effects variance is: 2.008254



Posterior means at iteration: 5000
Residual variance: 9.123002

Posterior means at iteration: 10000
Residual variance: 9.159126


running MCMC for BayesB... 50%|█████████████            |  ETA: 0:00:00

20 markers on 100 individuals were added.

The prior for marker effects variance is calculated from the genetic variance and π. The mean of the prior for the marker effects variance is: 4.016508




running MCMC for BayesB...100%|█████████████████████████| Time: 0:00:00



Posterior means at iteration: 5000
Residual variance: 9.561396
π: 0.758


...

running MCMC for BayesB... 66%|████████████████         |  ETA: 0:00:00


Posterior means at iteration: 10000
Residual variance: 11.570169
π: 0.573
20 markers on 100 individuals were added.

The prior for marker effects variance is calculated from the genetic variance and π. The mean of the prior for the marker effects variance is: 4.016508



Posterior means at iteration: 5000
Residual variance: 11.546987
Marker effects variance: 2.910386
π: 0.584


running MCMC for BayesB...100%|█████████████████████████| Time: 0:00:00



Posterior means at iteration: 10000
Residual variance: 11.545688
Marker effects variance: 2.863709
π: 0.578


running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:00


In [14]:
println("     True   BayesA BayesB BayesC RR-BLUP  OLS")
@printf("    %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resMeans[1],resMeans[2],resMeans[3],resMeans[4],resMeans[5],resMeans[6] )
@printf("+/- %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resStd[1],resStd[2],resStd[3],resStd[4],resStd[5],resStd[6] )

     True   BayesA BayesB BayesC RR-BLUP  OLS
     12.01  20.01  12.52  12.44  21.48  24.66
+/-   0.15   0.19   0.25   0.24   0.19   0.23


### Comparison of BayesA, BayesB, BayesCPi, RR-BLUP and OLS with Sample size n = 100 and number p = 1000 of markers and $\pi=0.95$

In [15]:
n  = 100
p  = 1000
probNull = 0.95

N  = 25
Vg = 20
Ve = 10

Random.seed!(4321)

Valpha = Vg/p*2
λ = Ve/Valpha
X = rand(Binomial(2,0.5), n,p)
mme = float(X'X + Identity(p)*λ)
mmei = inv(mme)
xpxi = inv(X'X + Identity(p)*.001)
CSV.write("genotypes",DataFrame([string.(1:n) X]), writeheader=false)

resSim = [getEstimates(X,mmei,xpxi,probNull,Vg, Ve) for i=1:N]
resMat = [row[i] for row in resSim, i=1:6]

resMeans = mean(resMat,dims=1)
resStd = std(resMat,dims=1)./N;

1000 markers on 100 individuals were added.

The prior for marker effects variance is calculated from the genetic variance and π. The mean of the prior for the marker effects variance is: 0.040202




running MCMC for BayesB... 43%|███████████              |  ETA: 0:00:03


Posterior means at iteration: 5000
Residual variance: 2.397599


running MCMC for BayesB... 88%|██████████████████████   |  ETA: 0:00:01


Posterior means at iteration: 10000
Residual variance: 2.360425


...

running MCMC for BayesC... 38%|█████████                |  ETA: 0:00:01


Posterior means at iteration: 5000
Residual variance: 4.924818
Marker effects variance: 0.036161
π: 0.436


running MCMC for BayesC... 84%|█████████████████████    |  ETA: 0:00:00


Posterior means at iteration: 10000
Residual variance: 4.773106
Marker effects variance: 0.040072
π: 0.463


running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:02


In [16]:
println("     True   BayesA BayesB BayesC RR-BLUP  OLS")
@printf("    %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resMeans[1],resMeans[2],resMeans[3],resMeans[4],resMeans[5],resMeans[6] )
@printf("+/- %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resStd[1],resStd[2],resStd[3],resStd[4],resStd[5],resStd[6] )

     True   BayesA BayesB BayesC RR-BLUP  OLS
     16.28  65.95  35.71  31.16  50.01  78.94
+/-   0.08   0.16   0.35   0.34   0.14   0.23


### Comparison of BayesA, BayesB, BayesCPi, RR-BLUP and OLS with Sample size n = 300 and number p = 1000 of markers and $\pi = 0.95$

In [17]:
n  = 300
p  = 1000
probNull = 0.95

N  = 25
Vg = 20
Ve = 10

Random.seed!(4321)

Valpha = Vg/p*2
λ = Ve/Valpha
X = rand(Binomial(2,0.5), n,p)
mme = float(X'X + Identity(p)*λ)
mmei = inv(mme)
xpxi = inv(X'X)
CSV.write("genotypes",DataFrame([string.(1:n) X]), writeheader=false)

resSim = [getEstimates(X,mmei,xpxi,probNull,Vg, Ve) for i=1:N]
resMat = [row[i] for row in resSim, i=1:6]

resMeans = mean(resMat,dims=1)
resStd = std(resMat,dims=1)./N;

1000 markers on 300 individuals were added.

The prior for marker effects variance is calculated from the genetic variance and π. The mean of the prior for the marker effects variance is: 0.040065




running MCMC for BayesB... 43%|███████████              |  ETA: 0:00:03


Posterior means at iteration: 5000
Residual variance: 2.427125


running MCMC for BayesB... 87%|██████████████████████   |  ETA: 0:00:01


Posterior means at iteration: 10000
Residual variance: 2.410644


...

running MCMC for BayesC... 41%|██████████               |  ETA: 0:00:02


Posterior means at iteration: 5000
Residual variance: 8.709031
Marker effects variance: 0.030461
π: 0.751


running MCMC for BayesC... 85%|█████████████████████    |  ETA: 0:00:00


Posterior means at iteration: 10000
Residual variance: 8.893261
Marker effects variance: 0.03333
π: 0.792


running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:03


In [18]:
println("     True   BayesA BayesB BayesC RR-BLUP  OLS")
@printf("    %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resMeans[1],resMeans[2],resMeans[3],resMeans[4],resMeans[5],resMeans[6] )
@printf("+/- %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resStd[1],resStd[2],resStd[3],resStd[4],resStd[5],resStd[6] )

     True   BayesA BayesB BayesC RR-BLUP  OLS
     15.10 115.51  20.16  21.64  85.46 3478.54
+/-   0.07   0.11   0.47   0.41   0.13  41.92


### Comparison of BayesA, BayesB, BayesCPi, RR-BLUP and OLS with Sample size n = 500 and number p = 1000 of markers and $\pi=0.95$

In [19]:
n  = 500
p  = 1000
probNull = 0.95

N  = 25
Vg = 20
Ve = 10

Random.seed!(4321)

Valpha = Vg/p*2
λ = Ve/Valpha
X = rand(Binomial(2,0.5), n,p)
mme = float(X'X + Identity(p)*λ)
mmei = inv(mme)
xpxi = inv(X'X)
CSV.write("genotypes",DataFrame([string.(1:n) X]), writeheader=false)

resSim = [getEstimates(X,mmei,xpxi,probNull,Vg, Ve) for i=1:N]
resMat = [row[i] for row in resSim, i=1:6]

resMeans = mean(resMat,dims=1)
resStd = std(resMat,dims=1)./N;

1000 markers on 500 individuals were added.

The prior for marker effects variance is calculated from the genetic variance and π. The mean of the prior for the marker effects variance is: 0.04004




running MCMC for BayesB... 43%|███████████              |  ETA: 0:00:05


Posterior means at iteration: 5000
Residual variance: 5.791029


running MCMC for BayesB... 89%|██████████████████████   |  ETA: 0:00:01


Posterior means at iteration: 10000
Residual variance: 5.733718


...

running MCMC for BayesC... 41%|██████████               |  ETA: 0:00:02


Posterior means at iteration: 5000
Residual variance: 9.591743
Marker effects variance: 0.03608
π: 0.949


running MCMC for BayesC... 86%|█████████████████████    |  ETA: 0:00:00


Posterior means at iteration: 10000
Residual variance: 9.56742
Marker effects variance: 0.035557
π: 0.942


running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:03


In [20]:
println("     True   BayesA BayesB BayesC RR-BLUP  OLS")
@printf("    %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resMeans[1],resMeans[2],resMeans[3],resMeans[4],resMeans[5],resMeans[6] )
@printf("+/- %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resStd[1],resStd[2],resStd[3],resStd[4],resStd[5],resStd[6] )

     True   BayesA BayesB BayesC RR-BLUP  OLS
     16.23 129.01  13.46  16.28 106.62 2102067.82
+/-   0.08   0.15   0.24   0.27   0.14 46690.09
